### Finviz Data Cleaning and Transformation Pipeline

This notebook processes the raw data downloaded from Finviz, cleans it, and transforms it into an analysis-ready format.

**Workflow:**

1.  **Load Data:** Reads a raw Finviz `.parquet` file for a specific date.
2.  **Feature Engineering:** Creates new composite columns (`Info`, `MktCap AUM`).
3.  **Data Type Conversion:**
    *   Converts currency strings (e.g., `1.5B`, `250K`) into numeric values in millions.
    *   Converts percentage strings (e.g., `12.5%`) into numeric values.
    *   Converts other object columns to their proper numeric types.
4.  **Final Processing:** Sorts the data by market capitalization, sets the `Ticker` as the index, and adds a `Rank` column.
5.  **Save & Verify:** Saves the cleaned DataFrame to a new `.parquet` file and verifies the saved file.

### Setup and Configuration

**This is the only cell you need to modify.** It contains all imports, paths, and lists of columns for processing.

In [1]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np

# --- Project Path Setup ---
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))

SRC_DIR = ROOT_DIR / 'src'
if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

# Import config and custom utils now that path is set
from config import DATE_STR, DOWNLOAD_DIR, DEST_DIR
import utils

# --- File Path Configuration ---
# Build paths using pathlib for cross-platform compatibility
SOURCE_PATH = Path(DOWNLOAD_DIR) / f'df_finviz_{DATE_STR}_stocks_etfs.parquet'
DEST_PATH = Path(DEST_DIR) / f'{DATE_STR}_df_finviz_stocks_etfs.parquet'

# --- Column Processing Configuration ---
# Define which columns need specific cleaning operations.

# Columns to combine into the 'Info' column
INFO_COLS = ["Sector", "Industry", "Single Category", "Asset Type"]

# Columns with abbreviated currency values (B, M, K) to be converted to millions
CURRENCY_COLS = [
    'Market Cap', 'AUM', 'Sales', 'Income', 'Outstanding', 'Float', 
    'Short Interest', 'Avg Volume', 'Flows 1M', 'Flows 3M', 'Flows YTD',
    'MktCap AUM' # This is the new column we create
]

# Other columns that are numeric but stored as strings (objects)
# Note: Percentage columns are detected automatically in Step 3.
OTHER_NUMERIC_COLS = [
    "No.", "P/E", "Fwd P/E", "PEG", "P/S", "P/B", "P/C", "P/FCF",
    "Book/sh", "Cash/sh", "Dividend TTM", "EPS", "EPS next Q", "Short Ratio",
    "Curr R", "Quick R", "LTDebt/Eq", "Debt/Eq", "Beta", "ATR", "RSI",
    "Employees", "Recom", "Rel Volume", "Volume", "Target Price",
    "Prev Close", "Open", "High", "Low", "Price", "Holdings"
]

# --- Notebook Setup ---
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
pd.set_option('display.width', 2500)
%load_ext autoreload
%autoreload 2

# --- Verification ---
print(f"Source file: {SOURCE_PATH}")
print(f"Destination file: {DEST_PATH}")
print(f"Processing for date: {DATE_STR}")

Source file: C:\Users\ping\Downloads\df_finviz_2025-08-08_stocks_etfs.parquet
Destination file: c:\Users\ping\Files_win10\python\py311\stocks\data\2025-08-08_df_finviz_stocks_etfs.parquet
Processing for date: 2025-08-08


### Step 1: Load Raw Data

Load the source Parquet file into a pandas DataFrame.

In [2]:
print(f"--- Step 1: Loading data from {SOURCE_PATH.name} ---")

try:
    df = pd.read_parquet(SOURCE_PATH, engine='pyarrow')
    print("Data loaded successfully.")
    df.info()
    display(df.head(3))
except FileNotFoundError:
    print(f"ERROR: Source file not found at {SOURCE_PATH}")
    df = None  # Ensure df is None if loading fails
except Exception as e:
    print(f"An error occurred during file loading: {e}")
    df = None

--- Step 1: Loading data from df_finviz_2025-08-08_stocks_etfs.parquet ---
Data loaded successfully.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Columns: 111 entries, No. to Tags
dtypes: object(111)
memory usage: 1.3+ MB


,No.,Ticker,Company,Index,Sector,Industry,Country,Exchange,Market Cap,P/E,Fwd P/E,PEG,P/S,P/B,P/C,P/FCF,Book/sh,Cash/sh,Dividend,Dividend TTM,Dividend Ex Date,Payout Ratio,EPS,EPS next Q,EPS This Y,EPS Next Y,EPS Past 5Y,EPS Next 5Y,Sales Past 5Y,Sales Q/Q,EPS Q/Q,EPS YoY TTM,Sales YoY TTM,Sales,Income,EPS Surprise,Revenue Surprise,Outstanding,Float,Float %,Insider Own,Insider Trans,Inst Own,Inst Trans,Short Float,Short Ratio,Short Interest,ROA,ROE,ROIC,Curr R,Quick R,LTDebt/Eq,Debt/Eq,Gross M,Oper M,Profit M,Perf Week,Perf Month,Perf Quart,Perf Half,Perf Year,Perf YTD,Beta,ATR,Volatility W,Volatility M,SMA20,SMA50,SMA200,50D High,50D Low,52W High,52W Low,52W Range,All-Time High,All-Time Low,RSI,Earnings,IPO Date,Optionable,Shortable,Employees,Change from Open,Gap,Recom,Avg Volume,Rel Volume,Volume,Target Price,Prev Close,Open,High,Low,Price,Change,Single Category,Asset Type,Expense,Holdings,AUM,Flows 1M,Flows% 1M,Flows 3M,Flows% 3M,Flows YTD,Flows% YTD,Return% 1Y,Return% 3Y,Return% 5Y,Tags
0,1041,NNNN,Anbio Biotechnology,-,Healthcare,Medical Instruments & Supplies,Germany,NASD,7.35B,842.57,-,-,830.18,121.30,603.21,3265.37,0.42,0.08,-,-,-,-,0.06,-,-,-,-,-,-,-36.00%,-267.38%,5.21%,21.97%,8.85M,2.56M,-,-,42.29M,39.69M,93.85%,9.57%,0.00%,0.15%,310.54%,0.44%,1.38,0.18M,13.83%,15.00%,14.40%,10.71,10.71,0.00,0.00,71.91%,24.34%,28.95%,19.30%,0.64%,670.14%,-,-,784.92%,-,4.94,10.48%,12.56%,10.47%,61.21%,195.45%,-8.25%,574.04%,-8.25%,885.71%,5.18 - 55.65,-8.25%,885.71%,63.28,-,2/19/2025,No,Yes,27,12.99%,-1.87%,-,127.38K,0.87,"110,715",-,46.05,45.19,53.12,45.40,51.06,10.88%,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
1,1042,ALSN,Allison Transmission Holdings Inc,-,Consumer Cyclical,Auto Parts,USA,NYSE,7.34B,9.96,9.26,0.76,2.29,4.19,9.43,11.30,20.96,9.30,1.23%,1.04,5/19/2025,12.04%,8.81,1.96,-0.59%,14.72%,11.09%,13.05%,3.63%,-0.25%,7.96%,15.17%,2.70%,3.20B,762.00M,3.85%,1.93%,83.62M,83.05M,99.32%,0.68%,-4.46%,101.25%,-1.05%,4.49%,4.48,3.73M,14.40%,47.08%,18.31%,3.36,2.64,1.37,1.38,48.22%,31.25%,23.81%,0.05%,-8.15%,-11.13%,-24.01%,4.68%,-18.81%,1.02,2.28,2.69%,2.50%,-0.38%,-6.43%,-14.35%,-16.73%,4.78%,-28.40%,9.13%,80.39 - 122.53,-28.40%,454.55%,43.29,Aug 04/a,3/15/2012,Yes,Yes,4000,-0.23%,0.59%,2.50,830.97K,0.81,"674,701",102.50,87.41,87.93,88.37,87.34,87.73,0.37%,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
2,1043,ZK,ZEEKR Intelligent Technology Holding Ltd. ADR,-,Consumer Cyclical,Auto Manufacturers,China,NYSE,7.33B,-,14.70,-,0.64,-,5.37,-,-12.89,5.39,-,-,-,-,-2.84,-0.78,35.34%,180.11%,-,-,118.66%,47.60%,66.34%,37.41%,43.08%,11.52B,-710.26M,-136.23%,19.27%,253.10M,-,-,-,-,3.82%,-3.45%,-,1.49,0.78M,-11.48%,-,-,0.59,0.45,-,-,17.93%,-7.36%,-6.16%,3.39%,6.67%,11.09%,7.22%,81.05%,2.01%,0.07,0.67,1.67%,2.44%,-0.21%,5.76%,11.28%,-4.64%,14.54%,-13.11%,122.69%,13.00 - 33.32,-13.11%,122.69%,57.87,Aug 14/b,5/10/2024,Yes,Yes,17439,0.14%,-0.10%,1.00,520.41K,0.30,"157,384",35.40,28.94,28.91,29.38,28.84,28.95,0.03%,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-


### Step 2: Feature Engineering - Create Composite Columns

Combine existing columns to create more meaningful features: `Info` and `MktCap AUM`.

In [3]:
if df is not None:
    print("\n--- Step 2: Engineering new features ---")
    
    # 1. Create 'Info' column by concatenating category columns.
    for col in INFO_COLS:
        if col in df.columns:
            df[col] = df[col].replace('-', '')
    df['Info'] = df[INFO_COLS].apply(lambda row: ', '.join(filter(None, row.astype(str))), axis=1)
    print("Created 'Info' column.")

    # 2. Create 'MktCap AUM' by concatenating 'Market Cap' and 'AUM'.
    # This combines stock and ETF liquidity metrics into a single string column for now.
    # It will be converted to numeric in the next step.
    df['MktCap AUM'] = df['Market Cap'].replace('-', '') + df['AUM'].replace('-', '')
    print("Created 'MktCap AUM' column.")

    # Display the new columns for verification
    display(df[['Ticker', 'Info', 'MktCap AUM']].head(3))


--- Step 2: Engineering new features ---
Created 'Info' column.
Created 'MktCap AUM' column.


,Ticker,Info,MktCap AUM
0,NNNN,"Healthcare, Medical Instruments & Supplies",7.35B
1,ALSN,"Consumer Cyclical, Auto Parts",7.34B
2,ZK,"Consumer Cyclical, Auto Manufacturers",7.33B


### Step 3: Data Type Conversion

This multi-part step cleans and converts all string-based numeric and percentage columns into proper numeric types.

#### Part A: Convert Abbreviated Currency Columns to Millions

In [4]:
def convert_to_millions(value: str) -> float:
    """Converts a string with a T/B/M/K suffix to a numeric value in millions."""
    if pd.isna(value):
        return np.nan
    
    value_str = str(value).strip().upper()
    if not value_str:
        return np.nan

    multipliers = {'T': 1_000_000, 'B': 1_000, 'M': 1, 'K': 0.001}
    suffix = value_str[-1]
    
    if suffix in multipliers:
        number_part = value_str[:-1]
        try:
            return float(number_part) * multipliers[suffix]
        except (ValueError, TypeError):
            return np.nan
    return np.nan

if df is not None:
    print("\n--- Step 3a: Converting currency columns to millions ---")
    new_names = {}
    for col in CURRENCY_COLS:
        if col in df.columns:
            df[col] = df[col].apply(convert_to_millions)
            new_names[col] = f"{col}, M"
    
    df.rename(columns=new_names, inplace=True)
    print(f"Converted and renamed {len(new_names)} columns.")
    display(df[[name for name in new_names.values() if name in df.columns]].head(3))


--- Step 3a: Converting currency columns to millions ---
Converted and renamed 12 columns.


,"Market Cap, M","AUM, M","Sales, M","Income, M","Outstanding, M","Float, M","Short Interest, M","Avg Volume, M","Flows 1M, M","Flows 3M, M","Flows YTD, M","MktCap AUM, M"
0,7350.0,NaN,8.85,2.56,42.29,39.69,0.18,0.12738,NaN,NaN,NaN,7350.0
1,7340.0,NaN,3200.00,762.00,83.62,83.05,3.73,0.83097,NaN,NaN,NaN,7340.0
2,7330.0,NaN,11520.00,-710.26,253.10,NaN,0.78,0.52041,NaN,NaN,NaN,7330.0


#### Part B: Convert Percentage Columns to Numeric

In [5]:
if df is not None:
    print("\n--- Step 3b: Converting percentage columns ---")
    percent_cols = [
        col for col in df.columns if df[col].dtype == 'object' and df[col].str.endswith('%', na=False).any()
    ]

    if not percent_cols:
        print("No new percentage columns found to modify.")
    else:
        print("Processing the following percentage columns:")
        for col in percent_cols:
            df[col] = pd.to_numeric(df[col].str.replace('%', ''), errors='coerce')
            new_name = f"{col} %" if '%' not in col else col
            df.rename(columns={col: new_name}, inplace=True)
            print(f"  - Converted '{col}' to numeric and renamed to '{new_name}'")


--- Step 3b: Converting percentage columns ---
Processing the following percentage columns:
  - Converted 'Dividend' to numeric and renamed to 'Dividend %'
  - Converted 'Payout Ratio' to numeric and renamed to 'Payout Ratio %'
  - Converted 'EPS This Y' to numeric and renamed to 'EPS This Y %'
  - Converted 'EPS Next Y' to numeric and renamed to 'EPS Next Y %'
  - Converted 'EPS Past 5Y' to numeric and renamed to 'EPS Past 5Y %'
  - Converted 'EPS Next 5Y' to numeric and renamed to 'EPS Next 5Y %'
  - Converted 'Sales Past 5Y' to numeric and renamed to 'Sales Past 5Y %'
  - Converted 'Sales Q/Q' to numeric and renamed to 'Sales Q/Q %'
  - Converted 'EPS Q/Q' to numeric and renamed to 'EPS Q/Q %'
  - Converted 'EPS YoY TTM' to numeric and renamed to 'EPS YoY TTM %'
  - Converted 'Sales YoY TTM' to numeric and renamed to 'Sales YoY TTM %'
  - Converted 'EPS Surprise' to numeric and renamed to 'EPS Surprise %'
  - Converted 'Revenue Surprise' to numeric and renamed to 'Revenue Surprise 

#### Part C: Convert Other String-Based Numeric Columns

In [6]:
if df is not None:
    print("\n--- Step 3c: Converting other numeric string columns ---")
    converted_count = 0
    for col in OTHER_NUMERIC_COLS:
        if col in df.columns and df[col].dtype == 'object':
            df[col] = pd.to_numeric(df[col].str.replace(',', '', regex=False), errors='coerce')
            converted_count += 1
            
    print(f"Converted {converted_count} additional columns to numeric type.")
    print("\nData types after all conversions:")
    df.info()


--- Step 3c: Converting other numeric string columns ---
Converted 32 additional columns to numeric type.

Data types after all conversions:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Columns: 113 entries, No. to MktCap AUM, M
dtypes: float64(94), int64(2), object(17)
memory usage: 1.3+ MB


### Step 4: Final Processing - Sort, Index, and Rank

Sort the DataFrame by the unified liquidity metric, set the `Ticker` as the index, and add a final `Rank`.

In [7]:
if df is not None:
    print("\n--- Step 4: Finalizing DataFrame ---")
    
    # 1. Sort by the primary metric in descending order
    df.sort_values(by='MktCap AUM, M', ascending=False, inplace=True, na_position='last')
    print("Sorted DataFrame by 'MktCap AUM, M'.")
    
    # 2. Add a 'Rank' column based on the new sort order
    df['Rank'] = range(1, len(df) + 1)
    print("Added 'Rank' column.")
    
    # 3. Set 'Ticker' as the index
    if 'Ticker' in df.columns:
        df.set_index('Ticker', inplace=True)
        print("Set 'Ticker' as the index.")
    
    print("\nFinal DataFrame structure:")
    display(df[['Rank', 'Info', 'MktCap AUM, M']].head())


--- Step 4: Finalizing DataFrame ---
Sorted DataFrame by 'MktCap AUM, M'.
Added 'Rank' column.
Set 'Ticker' as the index.

Final DataFrame structure:


C:\Users\ping\AppData\Local\Temp\ipykernel_9868\3337454481.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Rank'] = range(1, len(df) + 1)


,Rank,Info,"MktCap AUM, M"
Ticker,,,
NVDA,1,"Technology, Semiconductors",4457880.0
MSFT,2,"Technology, Software - Infrastructure",3880410.0
AAPL,3,"Technology, Consumer Electronics",3403640.0
GOOGL,4,"Communication Services, Internet Content & Inf...",2440010.0
GOOG,5,"Communication Services, Internet Content & Inf...",2440010.0


### Step 5: Save and Verify Cleaned Data

Save the final, cleaned DataFrame to a new Parquet file and read it back to verify integrity.

In [8]:
if df is not None:
    print("\n--- Step 5: Saving and verifying data ---")
    try:
        # Ensure destination directory exists
        DEST_PATH.parent.mkdir(parents=True, exist_ok=True)
        
        # Save the file
        df.to_parquet(DEST_PATH, engine='pyarrow', compression='zstd')
        print(f"Successfully saved cleaned data to: {DEST_PATH}")

        # Verify by loading it back
        loaded_df = pd.read_parquet(DEST_PATH, engine='pyarrow')
        print("\nVerification successful. First 20 rows of the saved file:")
        display(loaded_df.head(20))
        
    except Exception as e:
        print(f"An error occurred during save or verification: {e}")


--- Step 5: Saving and verifying data ---
Successfully saved cleaned data to: c:\Users\ping\Files_win10\python\py311\stocks\data\2025-08-08_df_finviz_stocks_etfs.parquet

Verification successful. First 20 rows of the saved file:


,No.,Company,Index,Sector,Industry,Country,Exchange,"Market Cap, M",P/E,Fwd P/E,PEG,P/S,P/B,P/C,P/FCF,Book/sh,Cash/sh,Dividend %,Dividend TTM,Dividend Ex Date,Payout Ratio %,EPS,EPS next Q,EPS This Y %,EPS Next Y %,EPS Past 5Y %,EPS Next 5Y %,Sales Past 5Y %,Sales Q/Q %,EPS Q/Q %,EPS YoY TTM %,Sales YoY TTM %,"Sales, M","Income, M",EPS Surprise %,Revenue Surprise %,"Outstanding, M","Float, M",Float %,Insider Own %,Insider Trans %,Inst Own %,Inst Trans %,Short Float %,Short Ratio,"Short Interest, M",ROA %,ROE %,ROIC %,Curr R,Quick R,LTDebt/Eq,Debt/Eq,Gross M %,Oper M %,Profit M %,Perf Week %,Perf Month %,Perf Quart %,Perf Half %,Perf Year %,Perf YTD %,Beta,ATR,Volatility W %,Volatility M %,SMA20 %,SMA50 %,SMA200 %,50D High %,50D Low %,52W High %,52W Low %,52W Range,All-Time High %,All-Time Low %,RSI,Earnings,IPO Date,Optionable,Shortable,Employees,Change from Open %,Gap %,Recom,"Avg Volume, M",Rel Volume,Volume,Target Price,Prev Close,Open,High,Low,Price,Change %,Single Category,Asset Type,Expense %,Holdings,"AUM, M","Flows 1M, M",Flows% 1M,"Flows 3M, M",Flows% 3M,"Flows YTD, M",Flows% YTD,Return% 1Y,Return% 3Y,Return% 5Y,Tags,Info,"MktCap AUM, M",Rank
Ticker,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
NVDA,1,NVIDIA Corp,"DJIA, NDX, S&P 500",Technology,Semiconductors,USA,NASD,4457880.0,58.85,31.08,1.86,30.02,53.14,83.03,61.86,3.44,2.20,0.02,0.04,6/11/2025,1.16,3.10,1.00,45.97,34.69,91.83,31.64,64.24,69.18,27.60,81.36,86.17,148510.0,76770.0,9.89,1.68,24390.00,23410.00,95.98,4.07,-0.69,66.28,0.24,0.86,1.05,201.66,75.89,115.46,81.82,3.39,2.96,0.12,0.12,70.11,58.03,51.69,5.17,11.33,55.66,41.98,75.25,36.05,2.11,4.33,2.45,2.30,4.63,14.75,34.47,-0.64,37.45,-0.64,110.92,86.62 - 183.88,-0.64,547999.96,69.70,Aug 27/a,1/22/1999,Yes,Yes,36000.0,0.62,0.44,1.38,191.28000,0.64,122424997,185.33,180.77,181.57,183.30,180.40,182.70,1.07,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,"Technology, Semiconductors",4457880.0,1
MSFT,2,Microsoft Corporation,"DJIA, NDX, S&P 500",Technology,Software - Infrastructure,USA,NASD,3880410.0,38.27,28.58,2.28,13.77,11.30,41.03,54.19,46.20,12.72,0.68,3.24,8/21/2025,24.34,13.64,3.65,13.68,17.78,18.80,16.76,14.52,18.10,23.77,15.50,14.93,281720.0,101830.0,8.16,3.40,7430.00,7320.00,98.52,1.47,-0.12,73.53,0.47,0.73,2.74,53.79,18.00,33.28,22.93,1.35,1.35,0.29,0.33,68.82,45.62,36.15,-0.39,4.10,19.14,25.54,30.64,23.85,1.05,9.39,1.67,1.55,1.28,5.32,19.86,-6.01,14.66,-6.01,51.41,344.79 - 555.45,-6.01,655081.84,59.69,Jul 30/a,3/13/1986,Yes,Yes,228000.0,-0.08,0.31,1.24,19.65000,0.79,15490364,618.66,520.84,522.44,524.66,519.41,522.04,0.23,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,"Technology, Software - Infrastructure",3880410.0,2
AAPL,3,Apple Inc,"DJIA, NDX, S&P 500",Technology,Consumer Electronics,USA,NASD,3403640.0,34.87,28.89,4.11,8.33,51.76,61.47,35.39,4.43,3.73,0.45,1.01,8/11/2025,16.11,6.58,1.75,8.93,7.97,15.41,8.49,8.51,9.63,12.19,0.15,5.97,408620.0,99280.0,9.18,4.99,14860.00,14830.00,99.79,0.10,-1.27,64.69,-0.58,0.63,1.71,93.95,29.94,149.81,66.96,0.87,0.83,1.25,1.54,46.68,31.87,24.30,13.33,7.98,16.13,-1.66,10.67,-8.41,1.10,5.44,3.35,2.07,8.41,10.98,3.73,3.85,17.57,-11.82,35.54,169.21 - 260.10,-11.82,360422.48,73.66,Jul 31/a,12/12/1980,Yes,Yes,164000.0,3.86,0.36,1.94,54.78000,2.03,111345109,233.97,220.03,220.82,231.00,219.25,229.35,4.24,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,"Technology, Consumer Electronics",3403640.0,3
GOOGL,5,Alphabet Inc,"NDX, S&P 500",Communication Services,Internet Content & Information,USA,NASD,2440010.0,21.45,18.97,1.48,6.57,6.72,25.64,36.57,29.98,7.85,0.26,0.81,9/8/2025,7.46,9.39,2.33,23.63,6.84,26.76,14.49,16.73,14.06,22.28,34.52,13.19,371210.0,115570.0,5.83,2.54,5820.00,5800.00,99.74,52.04,0.02,38.31,-1.71,1.00,1.43,58.23,25.21,34.83,28.86,1.90,1.90,0.10,0.11,58.92,32.65,31.13,6.50,13.40,30.55,5.13,27.25,6.40,1.00,4.28,2.14,2.03,5.41,11.45,14.55,1.75,24.33,-2.72,43.33,140.53 - 207.05,-2.72,8287.